In [74]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from scipy.optimize import differential_evolution
from loadAndPreprocess import load_and_preprocess

well_info, _ = load_and_preprocess()


def objective_function(new_wells, *args):
    existing_wells = args[0]
    all_wells = np.vstack((existing_wells, new_wells.reshape(-1, 2)))
    dist_mat = distance_matrix(all_wells, all_wells)
    np.fill_diagonal(dist_mat, np.inf)
    return -np.min(dist_mat)  # We return the negative because we want to maximize the minimum distance


def optimize_well_placement(existing_wells, num_new_wells):
    bounds = []
    x_min, y_min = np.min(existing_wells, axis=0)
    x_max, y_max = np.max(existing_wells, axis=0)

    # Extend bounds a bit if needed
    extension = 0.1 * (x_max - x_min)
    for _ in range(num_new_wells):
        bounds.extend([(x_min - extension, x_max + extension), (y_min - extension, y_max + extension)])

    result = differential_evolution(
        objective_function,
        bounds=bounds,
        args=(existing_wells,),
        strategy='best1bin',
        maxiter=1000,
        popsize=300,
        tol=0.001,
        mutation=(0.5, 1),
        recombination=0.7,
        seed=42
    )

    return result.x.reshape(-1, 2)


# Extract the existing well coordinates from the DataFrame
existing_wells = well_info[['X', 'Y']].values

# Number of new wells to add
num_new_wells = 5
new_wells = optimize_well_placement(existing_wells, num_new_wells)
print("Optimal locations for new wells:")
print(new_wells)

Optimal locations for new wells:
[[37701.21058232 47985.45425495]
 [35418.82555537 45875.12981296]
 [38308.92757506 45030.78541783]
 [34481.91374969 49406.42434986]
 [33722.57186987 46842.35711288]]


In [75]:
# Plot the existing and new wells using plotly
import plotly.express as px
fig = px.scatter(title='Existing Wells')
fig.add_scatter(x=existing_wells[:, 0], y=existing_wells[:, 1],
                mode='markers', marker=dict(color='#1f77b4'), name='Exsiting Wells')
fig.add_scatter(x=new_wells[:, 0], y=new_wells[:, 1], mode='markers', marker=dict(color='#d62728'), name='New Wells')
fig.update_layout(
    height=600,
    width=700,
)
fig.show()